<a href="https://colab.research.google.com/github/gabriellecoslope/developer-roadmap/blob/master/nivelamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nivelamento Computação - Curso de Verão em Bioinformática USP 2026
## Autor: Breno Tauil

Este notebook faz parte de uma simulação educacional do dia a dia de um **bioinformata em um hospital universitário**.


---


Você está de plantão na bioinformática clínica.  
Um alerta foi disparado: há indícios de um surto infeccioso em ambiente hospitalar. Amostras já foram coletadas, sequenciadas e os dados começaram a chegar. A equipe clínica espera respostas rápidas.

Seu trabalho começa **antes da análise**, **antes dos gráficos** e **antes dos resultados**. Ele começa no reconhecimento dos dados, na estimativa do impacto computacional e na escolha de estratégias que façam sentido **no tempo da clínica**, não apenas no tempo da máquina.

Os dados utilizados são reais e _downsampled_ de COVID-19. Os dados originais podem ser obtidos [aqui](https://www.ncbi.nlm.nih.gov/sra/SRX10525061)

---

- Leia as mensagens com atenção  
- Não pule etapas  
- Não presuma nada que você não tenha verificado  
- Se algo parece estranho nos dados, provavelmente é  
- Utilizaremos apenas a linha de comando

>IMPORTANTE
>Para rodar comandos de terminal no google collab, utilize a `!` antes do comando. Ex.:
```
!cat
```

---


**Assunto: Dados de sequenciamento**

De: Coordenação de Bioinformática Clínica

Para: Plantão de Bioinformática

Prioridade: Alta

---

Bom dia.

Os arquivos do sequenciamento já foram depositados no diretório do plantão. As amostras são todas do mesmo tipo de sequenciamento e, segundo a equipe do laboratório, passaram nos testes de qualidade. Isso nos poupa um tempo... Só preciso que você confirme se os dados entregues correspondem exatamente o solicitado, veja se os FASTQ estão com a estrutura correta. O técnico confirmou que os dados são todos Illumina. Preciso saber quantos dados realmente chegaram, o que eles representam e as amostras. Não esqueça de limpar os nomes.


Também preciso de uma estimativa do número de _reads_, temos que passar para a equipe clínica o tempo que isso vai levar para processar depois.

Por fim, faça uma varredura geral. Comentários, marcas internas, qualquer anotação que o laboratório tenha deixado nos arquivos pode conter informações operacionais importantes. Em situações anteriores, já perdemos tempo por ignorar detalhes aparentemente irrelevantes nessa etapa inicial.

Me retorne com checagem inicial o quanto antes, precisamos iniciar as análises antes do almoço.

—

Coordenação de Bioinformática Clínica

Hospital Universitário

### Definições:


---


#### Read (leitura)

Uma read é uma sequência curta de DNA gerada por uma máquina de sequenciamento.
Cada read representa um pequeno fragmento do material genético original.

* Um experimento gera milhares a milhões de reads

* Reads não são genes completos

* O volume de reads impacta diretamente o tempo computacional


---

#### FASTQ

Formato de arquivo padrão que armazena reads + qualidade.

Cada read ocupa 4 linhas:

* Identificador (começa com @)

* Sequência de bases (A, C, G, T)

* Separador (+)

* Qualidade (caracteres ASCII, representam o Phred Score)
  - Representa a confiança da máquina em cada base
  - `!` indica baixa qualidade
  - `I` indica alta qualidade
```
 Quality encoding: !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHI
                   |         |         |         |         |
    Quality score: 0........10........20........30........40   
```

Características importantes:

* Arquivo texto simples

* Pode ser inspecionado no terminal

No hospital, FASTQ é o dado bruto clínico.


---

#### Illumina

Empresa e tecnologia de sequenciamento amplamente usada.

No contexto do capítulo:

* “Illumina” indica tipo de máquina

* Todas as amostras virem da mesma plataforma reduz variabilidade

* Misturar plataformas complica análises futuras

---



#### Comandos Linux

`ls` - Lista arquivos e diretórios
```
ls .
```
Serve para:

  - Ver quantos arquivos chegaram

  - Identificar nomes estranhos



---


- `cat` - Mostra o conteúdo de um arquivo de texto inteiro.
```
cat arquivo.fastq
```
  - Use para arquivos pequenos ou inspeção rápida
  

---


`head` - Mostra apenas o início do arquivo.
```
head arquivo.fastq
```

Ideal para:

  - Confirmar formato FASTQ

  - Ver identificadores

  - Checar estrutura sem carregar tudo


---

`wc` (word count) - Conta linhas, palavras e caracteres.

Uso comum:
```
wc -l arquivo.fastq
wc arquivo.fastq
```
- Base para estimativa de volume de dados

`grep` - Busca padrões de texto dentro de arquivos.

Uso:
```
grep "ILLUMINA" *.fastq
```

Serve para:

- Confirmar plataforma

- Encontrar padrões
---

Podem ser utilizados juntos:
```
grep "ILLUMINA" filename | wc -l
```
Quantas linhas possuem o string ILLUMINA?

### Conhecendo os Dados

* Quais são os arquivos? Onde estão?
* Estão na estrutura certa?
* Quantas reads temos?
* Como está a qualidade das reads?



---

### Controle de Qualidade

Algumas bases tem reads de péssima qualidade, é necessário remover as sequências ruins.

---
`sed` - Substitui um prefixo por outro, de forma sistemática:

```
sed 's/padrao/substituicao/' arquivo.fastq
```
O comando `d` pode ser utilizado pelo sed para deletar uma linha que ele encontrou.
```
sed '/@/d' arquivo
```

O comando `sed N` avisa ao sed que ele leia a próxima linha. `N;N;N;` força o comando a processar 4 linhas de uma única vez.

Misturando tudo:
```
sed 'N;N;N;/@/d' arquivo
```
Deleta o "bloco" de 4 linhas em que o símbolo @ esteja presente.

O comando `p` "printa" a linha ou padrão encontrado pelo sed:
```
sed '1p' arquivo
```


---

Operador `>`

Um operador de redirecionamento, é utilizado para redirecionar a saída de um comando para um arquivo.

Ex.:
```
sed '/@/d' arquivo > arquivo2
```



---

Um motivo presente no código genético do SARS-CoV-2 é:

`TTACAAACATTGGCCG`

Se trata de uma região viral bem conservada e conhecida.

Qual comando podemos utilizar para encontrar esse motivo nas nossas sequencias?



---

### O genoma de referência

Um **genoma de referência** é uma sequência de DNA usada como **padrão de comparação**.  
Em vez de analisar cada read isoladamente, nós **comparamos as reads** geradas no sequenciamento com esse genoma já conhecido.

> “Em que parte do genoma essas sequências se encaixam?”
